In [4]:
%run plot.py

0.05999999865889549
processing data
doing ket/nq for ks
<class 'pandas.core.series.Series'>
doing ket/nq for lambda
<class 'pandas.core.series.Series'>
processing model
processing lc model
doing ket/nq for tamulc
doing ket/nq for tamulc
processing d0 model
doing ket/nq for tamud0
doing ket/nq for tamud0
processing lc model
doing ket/nq for catanialc
processing d0 model
doing ket/nq for cataniad0
processing lc model
doing ket/nq for langevinlc
processing d0 model
doing ket/nq for langevind0
processing lc model
doing ket/nq for htllc
processing d0 model
doing ket/nq for htld0
processing lc model
doing ket/nq for latQCDlc
processing d0 model
doing ket/nq for latQCDd0
processing lc model
doing ket/nq for lbtlc
processing d0 model
doing ket/nq for lbtd0
processing lc model
doing ket/nq for epos4hqlc
processing d0 model
doing ket/nq for epos4hqd0
processing data
doing pt/nq for ks
doing pt/nq for lambda
processing model
processing lc model
doing pt/nq for tamulc
doing pt/nq for tamulc
proces

Info in <TCanvas::Print>: pdf file ./performance.pdf has been created
Info in <TCanvas::Print>: pdf file ./compare-LF.pdf has been created
Info in <TCanvas::Print>: pdf file ./ncq.pdf has been created
Info in <TCanvas::Print>: pdf file ./ncq.pdf has been created
Info in <TCanvas::Print>: pdf file ./LcVsAllD_wTamu.pdf has been created
Info in <TCanvas::Print>: pdf file ./LcVsAllD_wTamu_woJpsi.pdf has been created
Info in <TCanvas::Print>: pdf file ./LcVsAllD_woTamu.pdf has been created
Info in <TCanvas::Print>: pdf file ./compare-model.pdf has been created









Fig1
1. use the same paper of the K0s and lambda for the pions, 
2. specify the centrality for the pion, i gues it is 3040, 
3. report systematic and statistical uncertainty for the pions too, 
4. the +- sign of the pion in the legend should stay above the pi not in front
Fig2
1. I would use 2 different black marker for D0 and Lc, 
and then I would use the same colors for the same model in the left and right panel 
otherwise is really difficult to understand how they behave of mesons and baryons, 
2. there is no need to report D0 and Lc for all the model and, 
if we uniform the colors, we can have just one model legend 
3. in the data/model it is not clear to me how you treated the systematic uncertainty
since you do not show them. 
4. I would consider adding the grid in the bottom panel to help driving the eye, 
5. the thick in the bottom panel are really difficult to see and 
in general they are not symmetric among the different panels, we should try to uniform, 
6. for the D0 I would start the x axis at 0 otherwise it is collapsed against the y axis 

git remote remove origin
git remote add origin git@github.com:dongfangqingxin/hcv2-prl-figures.git
git add .   
git commit -m 'no origin data'  
git push origin main    